In [13]:
from time import sleep
import serial


class Gripper:
    def __init__(self, usb_port):
        self.connect = serial.Serial(
            port=usb_port,
            baudrate=19200,
            parity=serial.PARITY_NONE,
            bytesize=8,
            stopbits=1,
            timeout=0.05,
            write_timeout=2.0,
        )
        print(self.connect)

    def close(self):
        self._write_word('010620400000', self.connect)  # clear position
        self._write_word('010620400028', self.connect)  # run position 2
    
    def open(self):
        self._write_word('010620400000', self.connect)  # clear position
        self._write_word('010620400018', self.connect)  # run position 1

    def close_abs(self):
        self._write_word('010620140064', self.connect)   # 速度
        self._write_word('0110200200020400001A90', self.connect)  # 位置 
        self._write_word('0106201E0001', self.connect)   # 啟動
    
    def open_abs(self):
        self._write_word('010620140064', self.connect)   # 速度
        self._write_word('0110200200020400001A90', self.connect)  # 位置 
        self._write_word('0106201E0001', self.connect)   # 啟動

    def origin(self):
        self._write_word('0106201E0003', self.connect)  # run origin

    def _write_word(self, code, connect):
        LRC = 255
        for i in range(len(code) // 2):
            s = code[i*2:i*2+2]
            s = int(s, 16)
            LRC -= s
        LRC += 1
        lrc_code = str(hex(LRC))[2:].upper()

        code = ':' + code + lrc_code + '\r\n'
        ret = connect.write(code.encode('ascii'))
        sleep(0.5)
        out = connect.read(1000)
        print('Input:', code)
        print('Output:', out)
        return out


In [14]:
g = Gripper('/dev/tty.usbserial-1420')

Serial<id=0x7f889b4e0990, open=True>(port='/dev/tty.usbserial-1420', baudrate=19200, bytesize=8, parity='N', stopbits=1, timeout=0.05, xonxoff=False, rtscts=False, dsrdtr=False)


In [20]:
g.close()

Input: :01062040000099

Output: b':01062040000099\r\n'
Input: :01062040002871

Output: b':01062040002871\r\n'


In [21]:
g.open()

Input: :01062040000099

Output: b':01062040000099\r\n'
Input: :01062040001881

Output: b':01062040001881\r\n'
